### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL

params = (
    f"fq={filter_query}&"
    f"sort={sort}&"
    f"fl={field_list}&"
    f"begin_date={begin_date}&"
    f"end_date={end_date}&"
    f"api-key={nyt_api_key}"
)

# Combine the base URL with the query parameters to build the full URL
full_url = url + params

In [4]:
# Create an empty list to store the reviews
reviews_list = []

# Loop through pages 0-19
for page in range(6):
    # Append the page number to the full URL
    current_url = f"{url}{params}&page={page}"

    # Make a "GET" request and retrieve the JSON
    response = requests.get(current_url)
    data = response.json()

    # Add a twelve-second interval between queries to stay within API query limits
    time.sleep(12)

    # Try and save the reviews to the reviews_list
    if 'response' in data and 'docs' in data['response']:
        articles = data['response']['docs']
        if articles:
            reviews_list.extend(articles)
            print(f"Checked page {page}")
        else:
            print(f"No results on page {page}. Breaking out of the loop.")
            break
    else:
        print(f"Error or no articles found on page {page}. Breaking out of the loop.")
        break

Checked page 0
Checked page 1
Checked page 2
Checked page 3
Checked page 4
Checked page 5


In [5]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_json = json.dumps(reviews_list[:5], indent=4)  # Convert to JSON and pretty-print
print(preview_json)

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

In [6]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews_list)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None


In [28]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early

df = pd.DataFrame(reviews_df)

# Applying the lambda function to create a new column 'title'
df['title'] = df['headline.main'].apply(
    lambda st: st[st.find("\u2018")+1:st.find("\u2019 Review")]
)

# Display the modified DataFrame
df[['headline.main', 'title']]

,headline.main,title
0,"‘The Attachment Diaries’ Review: Love, Sick",The Attachment Diaries
1,Review: ‘What’s Love Got to Do With It?’ Proba...,What’s Love Got to Do With It?’ Probably a Lo
2,‘You Can Live Forever’ Review: Do You Love Me ...,You Can Live Forever
3,‘A Tourist’s Guide to Love’ Review: A Wearying...,A Tourist’s Guide to Love
4,‘Other People’s Children’ Review: True Romance,Other People’s Children
5,‘One True Loves’ Review: A Romance Lost at Sea,One True Loves
6,‘The Lost Weekend: A Love Story’ Review: When ...,The Lost Weekend: A Love Story
7,‘A Thousand and One’ Review: A New York Love S...,A Thousand and One
8,‘Your Place or Mine’ Review: Try Neither,Your Place or Mine
9,‘Love in the Time of Fentanyl’ Review: Heartbr...,Love in the Time of Fentanyl


In [26]:
# Display the entire modified DataFrame to look like the sample output
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,None
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,"[{'name': 'subject', 'value': 'Documentary Fil...",2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,"[{'name': 'subject', 'va

In [35]:
# Function to convert string to JSON and then extract 'name' and 'value'
def extract_keywords(keyword_string):
    try:
        keyword_list = json.loads(keyword_string)  # Parse the string to a list of dictionaries using json.loads
        extracted_keywords = ""
        for item in keyword_list:
            if 'name' in item and 'value' in item:  # Ensure both keys exist before accessing
                keyword = f"{item['name']}: {item['value']};"
                extracted_keywords += keyword
        return extracted_keywords
    except json.JSONDecodeError:
        return "Invalid format"  # Return this or handle the exception as needed

# Apply the function to the 'keywords' column to fix it by converting from a string to a formatted string
df['keywords'] = df['keywords'].apply(extract_keywords)

# Display the modified DataFrame to verify the transformation
df

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,Invalid format,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
1,https://www.nytimes.com/2023/05/04/movies/what...,Two childhood friends navigate cultural differ...,The New York Times,Invalid format,2023-05-04T17:16:45+0000,287,Review: ‘What’s Love Got to Do With It?’ Proba...,None,None,What’s Love Got to Do With It?,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,What’s Love Got to Do With It?’ Probably a Lo
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,Invalid format,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,Invalid format,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist’s Guide to Love
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,Invalid format,2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People’s Children
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,Invalid format,2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves
6,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,Invalid format,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,None,None,None,By Glenn Kenny,"[{'firstname': 'Glenn', 'middlename': None, 'l...",None,The Lost Weekend: A Love Story
7,https://www.nytimes.com/2023/03/30/movies/a-th...,A mesmerizing Teyana Taylor stars in A.V. Rock...,The New York Times,Invalid format,2023-03-30T18:53:42+0000,971,‘A Thousand and One’ Review: A New York Love S...,Critic’s Pick,None,An Unbending Will Meets a Shifting City,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,A Thousand and One
8,https://www.nytimes.com/2023/02/09/movies/your...,This humdrum Netflix romantic comedy features ...,The New York Times,Invalid format,2023-02-10T00:00:05+0000,569,‘Your Place or Mine’ Review: Try Neither,None,None,They Have a Humdrum Kind of Love,None,None,None,By Amy Nicholson,"[{'firstname': 'Amy', 'middlename': None, 'las...",None,Your Place or Mine
9,https://www.nytimes.com/2023/02/02/movies/love...,"To combat the overdose crisis, a group that in...",The New York Times,Invalid format,2023-02-02T12:00:11+0000,306,‘Love in the Time of Fentanyl’ Review: Heartbr...,None,None,Love in the Time Of Fentanyl,None,None,None,By Concepción de León,"[{'firstname': 'Concepción', 'middlename': Non...",None,Love in the Time of Fentanyl


In [36]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
title_list = df['title'].to_list()
title_list

['The Attachment Diaries',
 'What’s Love Got to Do With It?’ Probably a Lo',
 'You Can Live Forever',
 'A Tourist’s Guide to Love',
 'Other People’s Children',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley’s Lover',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Clo

### Access The Movie Database API

In [10]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [11]:
# Create an empty list to store the results


# Create a request counter to sleep the requests after a multiple
# of 50 requests


# Loop through the titles

    # Check if we need to sleep before making a request


    # Add 1 to the request counter

    
    # Perform a "GET" request for The Movie Database


    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.

        # Get movie id


        # Make a request for a the full movie details


        # Execute "GET" request with url

        
        # Extract the genre names into a list


        # Extract the spoken_languages' English name into a list


        # Extract the production_countries' name into a list


        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list

        
        # Print out the title that was found

tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple of 50 requests
request_counter = 0

# Dummy list of movie titles for the sake of the example
movie_titles = ["Inception", "Avatar", "The Matrix", "Interstellar", "Gladiator"]

# Loop through the titles
for title in movie_titles:
    # Check if we need to sleep before making a request
    if request_counter and request_counter % 50 == 0:
        print("Sleeping for 10 seconds to avoid hitting API rate limit")
        time.sleep(10)  # Sleep for 10 seconds to respect rate limits

    # Add 1 to the request counter
    request_counter += 1

    # Perform a "GET" request for The Movie Database
    search_response = requests.get(url + title + tmdb_key_string)
    search_data = search_response.json()

    try:
        # Get movie id from the first search result
        movie_id = search_data['results'][0]['id']

        # Make a request for the full movie details
        details_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}&append_to_response=credits"
        details_response = requests.get(details_url)
        details_data = details_response.json()

        # Extract the genre names into a list
        genres = [genre['name'] for genre in details_data.get('genres', [])]

        # Extract the spoken_languages' English name into a list
        languages = [lang['english_name'] for lang in details_data.get('spoken_languages', [])]

        # Extract the production_countries' name into a list
        countries = [country['name'] for country in details_data.get('production_countries', [])]

        # Add the relevant data to a dictionary and append it to the tmdb_movies_list list
        movie_info = {
            'title': details_data['title'],
            'genres': genres,
            'languages': languages,
            'countries': countries
        }
        tmdb_movies_list.append(movie_info)

        # Print out the title that was found
        print(f"Found and added details for {title}")

    except IndexError:
        # This handles cases where the movie might not be found or the data is incomplete
        print(f"Movie not found or incomplete data for {title}")
    except KeyError:
        # This handles unexpected missing fields in the data
        print(f"Unexpected data structure while processing {title}")

# Optionally print or process the collected movie details list
print(tmdb_movies_list)


Found and added details for Inception
Found and added details for Avatar
Found and added details for The Matrix
Found and added details for Interstellar
Found and added details for Gladiator
[{'title': 'Inception', 'genres': ['Action', 'Science Fiction', 'Adventure'], 'languages': ['English', 'French', 'Japanese', 'Swahili'], 'countries': ['United Kingdom', 'United States of America']}, {'title': 'Avatar', 'genres': ['Action', 'Adventure', 'Fantasy', 'Science Fiction'], 'languages': ['English', 'Spanish'], 'countries': ['United States of America', 'United Kingdom']}, {'title': 'The Matrix', 'genres': ['Action', 'Science Fiction'], 'languages': ['English'], 'countries': ['United States of America']}, {'title': 'Interstellar', 'genres': ['Adventure', 'Drama', 'Science Fiction'], 'languages': ['English'], 'countries': ['United Kingdom', 'United States of America']}, {'title': 'Gladiator', 'genres': ['Action', 'Drama', 'Adventure'], 'languages': ['English'], 'countries': ['United Kingdom',

In [12]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
preview_json = json.dumps(tmdb_movies_list[:5], indent=4)
print(preview_json)

[
    {
        "title": "Inception",
        "genres": [
            "Action",
            "Science Fiction",
            "Adventure"
        ],
        "languages": [
            "English",
            "French",
            "Japanese",
            "Swahili"
        ],
        "countries": [
            "United Kingdom",
            "United States of America"
        ]
    },
    {
        "title": "Avatar",
        "genres": [
            "Action",
            "Adventure",
            "Fantasy",
            "Science Fiction"
        ],
        "languages": [
            "English",
            "Spanish"
        ],
        "countries": [
            "United States of America",
            "United Kingdom"
        ]
    },
    {
        "title": "The Matrix",
        "genres": [
            "Action",
            "Science Fiction"
        ],
        "languages": [
            "English"
        ],
        "countries": [
            "United States of America"
        ]
    },
    {
       

In [13]:
# Convert the results to a DataFrame
movies_df = pd.DataFrame(tmdb_movies_list)
movies_df.head()

,title,genres,languages,countries
0,Inception,"[Action, Science Fiction, Adventure]","[English, French, Japanese, Swahili]","[United Kingdom, United States of America]"
1,Avatar,"[Action, Adventure, Fantasy, Science Fiction]","[English, Spanish]","[United States of America, United Kingdom]"
2,The Matrix,"[Action, Science Fiction]",[English],[United States of America]
3,Interstellar,"[Adventure, Drama, Science Fiction]",[English],"[United Kingdom, United States of America]"
4,Gladiator,"[Action, Drama, Adventure]",[English],"[United Kingdom, United States of America]"


### Merge and Clean the Data for Export

In [14]:
# Merge the New York Times reviews and TMDB DataFrames on title


In [15]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing


# Create a list of characters to remove


# Loop through the list of columns to fix

    # Convert the column to type 'str'


    # Loop through characters to remove


# Display the fixed DataFrame


In [16]:
# Drop "byline.person" column


In [17]:
# Delete duplicate rows and reset index


In [18]:
# Export data to CSV without the index
